# Text Summarization. Homework

Всем привет! Это домашка по суммаризации текста.

На семинаре мы рассмотрели базовые модели для суммаризации текста. Попробуйте теперь улучшить два метода: TextRank и Extractive RNN. Задание достаточно большое и требует хорошую фантазию, тут можно эксперементировать во всю.

Для сдачи заданий надо получить определенное качество по test-у:

- 1 задание: 0.27 BLEU
- 2 задание: 0.3 BLEU

Если ваш подход пробивает это качество – задание считается пройденным. Плюсом будет описание того, почему вы решили использовать то или иное решение. 

Датасет: gazeta.ru

**P.S.** Возможно, в датасете находятся пустые данные. Проверьте эту гипотезу, и если надо, сделайте предобратоку датасета.


`Ноутбук создан на основе семинара Гусева Ильи на кафедре компьютерной лингвистики МФТИ.`

Загрузим датасет и необходимые библиотеки

In [0]:
!wget -q https://www.dropbox.com/s/43l702z5a5i2w8j/gazeta_train.txt
!wget -q https://www.dropbox.com/s/k2egt3sug0hb185/gazeta_val.txt
!wget -q https://www.dropbox.com/s/3gki5n5djs9w0v6/gazeta_test.txt

In [0]:
!pip install --upgrade razdel allennlp torch fasttext OpenNMT-py networkx pymorphy2 nltk rouge==0.3.1 summa
!pip install transformers youtokentome catalyst

In [0]:
import razdel

In [0]:
import random
import pandas as pd

def read_gazeta_records(file_name, shuffle=True, sort_by_date=False):
    assert shuffle != sort_by_date
    records = []
    with open(file_name, "r") as r:
        for line in r:
            records.append(eval(line)) # Simple hack
    records = pd.DataFrame(records)
    if sort_by_date:
        records = records.sort("date")
    if shuffle:
        records = records.sample(frac=1)
    return records

In [0]:
train_records = read_gazeta_records("gazeta_train.txt")
val_records = read_gazeta_records("gazeta_val.txt")
test_records = read_gazeta_records("gazeta_test.txt")

In [0]:
train_records.iloc[0]

url        https://www.gazeta.ru/sport/2018/12/27/a_12111...
text       Канадский тренер Брайан Орсер дал большое инте...
title      «Ты больше не сверхчеловек»: что Орсер изменил...
summary    Российская фигуристка Евгения Медведева уехала...
date                                     2018-12-28 08:18:36
Name: 17002, dtype: object

## 1 задание: TextRank (порог: 0.27 BLEU)

TextRank - unsupervised метод для составления кратких выжимок из текста. 
Описание метода:

1. Сплитим текст по предложениям
2. Считаем "похожесть" предложений между собой
3. Строим граф предложений с взвешенными ребрами
4. С помощью алгоритм PageRank получаем наиболее важные предложения, на основе которых делаем summary.

Функция похожести можно сделать и из нейросетевых(или около) моделек: FastText, ELMO и BERT. Выберете один метод, загрузите предобученную модель и с ее помощью для каждого предложениия сделайте sentence embedding. С помощью косинусной меры определяйте похожесть предложений.

Предобученные модели можно взять по [ссылке](http://docs.deeppavlov.ai/en/master/features/pretrained_vectors.html).

В общем, тут мы просто используем ру-Берт от диппавлов 

In [0]:
import transformers

In [2]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [0]:
from transformers import BertTokenizer, BertModel, AutoTokenizer, AutoModelWithLMHead

In [0]:
tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')
model = BertModel.from_pretrained("DeepPavlov/rubert-base-cased").to(device)

In [0]:
for param in model.parameters():
  param.requires_grad = False

In [0]:
from nltk.translate.bleu_score import corpus_bleu
from rouge import Rouge

def calc_scores(references, predictions, metric="all"):
    print("Count:", len(predictions))
    print("Ref:", references[-1])
    print("Hyp:", predictions[-1])

    if metric in ("bleu", "all"):
        print("BLEU: ", corpus_bleu([[r] for r in references], predictions))
    if metric in ("rouge", "all"):
        rouge = Rouge()
        scores = rouge.get_scores(predictions, references, avg=True)
        print("ROUGE: ", scores)

In [0]:
from itertools import combinations
import networkx as nx
import numpy as np
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [17]:
# Используйте эту штуку как бэйзлайн
def unique_words_similarity(words1, words2):
    '''
    Функция подсчёта близости предложений на основе пересечения слов
    ''' 
    words1 = set(words1)
    words2 = set(words2)
    if not len(words1) or not len(words2):
        return 0.0
    return len(words1.intersection(words2))/(np.log10(len(words1)) + np.log10(len(words2)))

def your_super_words_similarity(words1, words2, model, tokenizer):
    from sklearn.metrics.pairwise import cosine_similarity
    input_ids1 = torch.tensor(tokenizer.encode(words1, max_length=100, pad_to_max_length=True)).unsqueeze(0).to(device)  # Batch size 1
    outputs1 = model(input_ids1)
    last_hidden_states1 = outputs1[0].view(1,-1)
    input_ids2 = torch.tensor(tokenizer.encode(words2, max_length=100, pad_to_max_length=True)).unsqueeze(0).to(device)  # Batch size 1
    outputs2 = model(input_ids2)
    last_hidden_states2 = outputs2[0].view(1,-1)
    cos = torch.nn.CosineSimilarity(dim=0)
    output = cos(last_hidden_states1, last_hidden_states2)
    output = output.cpu().detach().numpy()
    if output[0] < 0:
      return 0
    else:
      return output[0]



def gen_text_rank_summary(text, tokenizer, model, calc_similarity=your_super_words_similarity, summary_part=0.1,  lower=False, morph=None):
    '''
    Составление summary с помощью TextRank
    '''
    # Разбиваем текст на предложения
    sentences = [sentence.text for sentence in razdel.sentenize(text)]
    n_sentences = len(sentences)

    if calc_similarity==unique_words_similarity:
    # Токенизируем предложения
      sentences_words = [[token.text.lower() if lower else token.text for token in razdel.tokenize(sentence)] for sentence in sentences]

    # При необходимости лемматизируем слова
      if morph is not None:
        sentences_words = [[morph.parse(word)[0].normal_form for word in words] for words in sentences_words]
      
      pairs = combinations(range(n_sentences), 2)
      scores = [(i, j, calc_similarity(sentences_words[i], sentences_words[j])) for i, j in pairs]

    else:
      # Для каждой пары предложений считаем близость
      pairs = combinations(range(n_sentences), 2)
      scores = [(i, j, calc_similarity(sentences[i], sentences[j], model=model, tokenizer=tokenizer)) for i, j in pairs]

    # Строим граф с рёбрами, равными близости между предложениями
    g = nx.Graph()
    g.add_weighted_edges_from(scores)

    # Считаем PageRank
    pr = nx.pagerank(g, max_iter=100)
    result = [(i, pr[i], s) for i, s in enumerate(sentences) if i in pr]
    result.sort(key=lambda x: x[1], reverse=True)

    # Выбираем топ предложений
    n_summary_sentences = max(int(n_sentences * summary_part), 1)
    result = result[:n_summary_sentences]

    # Восстанавливаем оригинальный их порядок
    result.sort(key=lambda x: x[0])

    # Восстанавливаем текст выжимки
    predicted_summary = " ".join([sentence for i, proba, sentence in result])
    predicted_summary = predicted_summary.lower() if lower else predicted_summary
    return predicted_summary

def calc_text_rank_score(records, calc_similarity=your_super_words_similarity, summary_part=0.1, lower=False, nrows=100, morph=None):
    references = []
    predictions = []

    for text, summary in records[['text', 'summary']].values[:nrows]:
        summary = summary if not lower else summary.lower()
        references.append(summary)

        predicted_summary = gen_text_rank_summary(text, tokenizer, model, calc_similarity, summary_part, lower, morph=morph)
        text = text if not lower else text.lower()
        predictions.append(predicted_summary)

    calc_scores(references, predictions)

#случайно дублировала вызов функции
calc_text_rank_score(test_records, calc_similarity=unique_words_similarity, morph=morph)
calc_text_rank_score(test_records, calc_similarity=your_super_words_similarity, morph=morph)

Count: 100
Ref: Генеральный директор Российского антидопингового агентства Юрий Ганус рассказал о санкциях, которые могут быть применены по отношению к сборной России по футболу в свете очередного допингового скандала в отечественном спорте. Так, по словам главы РУСАДА, существует вероятность того, что команда Станислава Черчесова может остаться без чемпионата Европы — 2020, а Санкт-Петербург будет исключен из списка городов-хозяев континентального первенства.
Hyp: Напомним, что разговоры о пропуске чемпионата Европы начались 25 ноября, когда комитет по соответствию WADA рекомендовал исполкому организации отстранить Россию от международных соревнований на четыре года (изначально страной «нон-грата» Россия становилась бы на Олимпийских играх 2020 и 2022 годов, чемпионате мира по футболу в Катаре – 2022 — о Евро ходили лишь слухи). Так, отечественные спортсмены будут вынуждены выступать под нейтральным флагом на всех международных турнирах до 2022 года, а все завоеванные там награды буду

## 2 Задание: Extractive RNN (порог: 0.3 BLEU)

Второй метод, который вам предлагается улучшить – поиск предложений для summary с помощью RNN. В рассмотренной методе мы использовали LSTM для генерации sentence embedding. Попробуйте использовать другие архитектуры: CNN, Transformer; или добавьте предобученные модели, как и в первом задании.

P.S. Тут предполагается, что придется изменять много кода в ячееках (например, поменять токенизацию). 

### Модель

Картинка для привлечения внимания:

![img](https://storage.googleapis.com/groundai-web-prod/media%2Fusers%2Fuser_14%2Fproject_398421%2Fimages%2Farchitecture.png)

Статья с оригинальным методом:
https://arxiv.org/pdf/1611.04230.pdf

Список вдохновения: 
- https://towardsdatascience.com/understanding-how-convolutional-neural-network-cnn-perform-text-classification-with-word-d2ee64b9dd0b Пример того, как можно применять CNN в текстовых задачах
- https://arxiv.org/pdf/1808.08745.pdf Очень крутой метод генерации summary без Transformers
- https://towardsdatascience.com/super-easy-way-to-get-sentence-embedding-using-fasttext-in-python-a70f34ac5b7c – простой метод генерации sentence embedding
- https://towardsdatascience.com/fse-2b1ffa791cf9 – Необычный метод генерации sentence embedding
- https://github.com/UKPLab/sentence-transformers – BERT предобученный для sentence embedding

P.S. Выше написанные ссылки нужны только для разогрева вашей фантазии, можно воспользоваться ими, а можно придумать свой.

Комментарий к заданию:
Если посмотреть на архитектуру ~~почти~~ SummaRuNNer, то в ней есть два главных элемента: первая часть, которая читает предложения и возвращает векторы на каждое предложение, и вторая, которая выбирает предложения для суммаризации. Вторую часть мы не трогаем, а первую меняем. На что меняем – как вы решите. Главное: она должна иметь хорошее качество и встроиться в текущую модель.

In [0]:
import copy
import random

def build_oracle_summary_greedy(text, gold_summary, calc_score, lower=True, max_sentences=30):
    '''
    Жадное построение oracle summary
    '''
    gold_summary = gold_summary.lower() if lower else gold_summary
    # Делим текст на предложения
    sentences = [sentence.text.lower() if lower else sentence.text for sentence in razdel.sentenize(text)][:max_sentences]
    n_sentences = len(sentences)
    oracle_summary_sentences = set()
    score = -1.0
    summaries = []
    for _ in range(min(n_sentences, 2)):
        for i in range(n_sentences):
            if i in oracle_summary_sentences:
                continue
            current_summary_sentences = copy.copy(oracle_summary_sentences)
            # Добавляем какое-то предложения к уже существующему summary
            current_summary_sentences.add(i)
            current_summary = " ".join([sentences[index] for index in sorted(list(current_summary_sentences))])
            # Считаем метрики
            current_score = calc_score(current_summary, gold_summary)
            summaries.append((current_score, current_summary_sentences))
        # Если получилось улучшить метрики с добавлением какого-либо предложения, то пробуем добавить ещё
        # Иначе на этом заканчиваем
        best_summary_score, best_summary_sentences = max(summaries)
        if best_summary_score <= score:
            break
        oracle_summary_sentences = best_summary_sentences
        score = best_summary_score
    oracle_summary = " ".join([sentences[index] for index in sorted(list(oracle_summary_sentences))])
    return oracle_summary, oracle_summary_sentences

def calc_single_score(pred_summary, gold_summary, rouge):
    return rouge.get_scores([pred_summary], [gold_summary], avg=True)['rouge-2']['f']

In [14]:
from tqdm import tqdm_notebook as tqdm

def calc_oracle_score(records, nrows=100, lower=True):
    references = []
    predictions = []
    rouge = Rouge()
  
    for text, summary in tqdm(records[['text', 'summary']].values[:nrows]):
        summary = summary if not lower else summary.lower()
        references.append(summary)
        predicted_summary, _ = build_oracle_summary_greedy(text, summary, calc_score=lambda x, y: calc_single_score(x, y, rouge))
        predictions.append(predicted_summary)

    calc_scores(references, predictions)

calc_oracle_score(test_records)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  



Count: 100
Ref: зарубежные спецслужбы намеренно ищут уязвимости в российском it-секторе, чтобы проводить масштабные кибератаки, заявил секретарь совбеза николай патрушев. по его словам, основные цели злоумышленников – объекты критической информационной инфраструктуры рф. эти атаки — за год несколько миллионов случаев — создают угрозу национальной безопасности.
Hyp: об этом заявил секретарь совета безопасности рф николай патрушев , передает риа «новости». «основными целями для оказания вредоносного воздействия остаются объекты критической информационной инфраструктуры россии, что создаст реальные угрозы национальной безопасности», – подчеркнул он.
BLEU:  0.40210244374874193
ROUGE:  {'rouge-1': {'f': 0.3384520074772536, 'p': 0.42678621721942234, 'r': 0.29817492526181794}, 'rouge-2': {'f': 0.1962514492944409, 'p': 0.2526423844827457, 'r': 0.17216638840714324}, 'rouge-l': {'f': 0.2850710736576467, 'p': 0.39215959707057324, 'r': 0.27366561891414426}}


## (!)
Если надо, поменяйте код загрузки токенизатора

Токенизатор нам и не нужен

In [0]:
# import os

# import youtokentome as yttm

# def train_bpe(records, model_path, model_type="bpe", vocab_size=10000, lower=True):
#     temp_file_name = "temp.txt"
#     with open(temp_file_name, "w") as temp:
#         for text, summary in records[['text', 'summary']].values:
#             if lower:
#                 summary = summary.lower()
#                 text = text.lower()
#             if not text or not summary:
#                 continue
#             temp.write(text + "\n")
#             temp.write(summary + "\n")
#     yttm.BPE.train(data=temp_file_name, vocab_size=vocab_size, model=model_path)

# train_bpe(train_records, "BPE_model.bin")

In [42]:
# import youtokentome as yttm

# bpe_processor = yttm.BPE('BPE_model.bin')
# bpe_processor.encode(["октябрь богат на изменения"], output_type=yttm.OutputType.SUBWORD)

[['▁окт', 'ябрь', '▁бога', 'т', '▁на', '▁изменения']]

## (!)
Если надо, поменяйте код словаря

Тоже не нужно

In [0]:
# from collections import Counter
# from typing import List, Tuple
# import os

# class Vocabulary:
#     def __init__(self, bpe_processor):
#         self.index2word = bpe_processor.vocab()
#         self.word2index = {w: i for i, w in enumerate(self.index2word)}
#         self.word2count = Counter()

#     def get_pad(self):
#         return self.word2index["<PAD>"]

#     def get_sos(self):
#         return self.word2index["<SOS>"]

#     def get_eos(self):
#         return self.word2index["<EOS>"]

#     def get_unk(self):
#         return self.word2index["<UNK>"]
    
#     def has_word(self, word) -> bool:
#         return word in self.word2index

#     def get_index(self, word):
#         if word in self.word2index:
#             return self.word2index[word]
#         return self.get_unk()

#     def get_word(self, index):
#         return self.index2word[index]

#     def size(self):
#         return len(self.index2word)

#     def is_empty(self):
#         empty_size = 4
#         return self.size() <= empty_size

#     def reset(self):
#         self.word2count = Counter()
#         self.index2word = ["<pad>", "<sos>", "<eos>", "<unk>"]
#         self.word2index = {word: index for index, word in enumerate(self.index2word)}

In [0]:
# vocabulary = Vocabulary(bpe_processor)
# vocabulary.size()

In [46]:
# ext_train_records.iloc[0]

url                 https://www.gazeta.ru/business/2018/08/27/1192...
text                Потери Киева в случае отказа России транспорти...
title               Развал страны: что ждет Украину без российског...
summary             В Киеве подсчитали потери от отказа России тра...
date                                              2018-08-27 19:43:33
sentences           [потери киева в случае отказа россии транспорт...
oracle_sentences                                              [0, 12]
oracle_summary      потери киева в случае отказа россии транспорти...
Name: 41718, dtype: object

In [15]:
from rouge import Rouge
import razdel

def add_oracle_summary_to_records(records, max_sentences=30, lower=True, nrows=1000):
    rouge = Rouge()
    sentences_ = []
    oracle_sentences_ = []
    oracle_summary_ = []
    records = records.iloc[:nrows].copy()

    for text, summary in tqdm(records[['text', 'summary']].values):
        summary = summary.lower() if lower else summary
        sentences = [sentence.text.lower() if lower else sentence.text for sentence in razdel.sentenize(text)][:max_sentences]
        oracle_summary, sentences_indicies = build_oracle_summary_greedy(text, summary, calc_score=lambda x, y: calc_single_score(x, y, rouge),
                                                                         lower=lower, max_sentences=max_sentences)
        sentences_ += [sentences]
        oracle_sentences_ += [list(sentences_indicies)]
        oracle_summary_ += [oracle_summary]
    records['sentences'] = sentences_
    records['oracle_sentences'] = oracle_sentences_
    records['oracle_summary'] = oracle_summary_
    return records

ext_train_records = add_oracle_summary_to_records(train_records, nrows=4096)
ext_val_records = add_oracle_summary_to_records(val_records, nrows=256)
ext_test_records = add_oracle_summary_to_records(test_records, nrows=256)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # This is added back by InteractiveShellApp.init_path()


## (!)
Если надо, поменяйте код генератора датасета и батчевалки

Что-то я тут пробовала мудрить с Бертом - но то памяти не хватало, то обучалось плохо... В итоге идем элементарнейшим путем - берем предобученные эмбедосы и пихаем их в линейные слои, сверху еще софтмакс. Сам фрагмент YourSentenceEncoder здесь и не нужен - оставила, так как, честно говоря, не хватило времени, сессия все дела. Результат посредственный, но бейзлайн пробивает. Времени улучшать ну совсем нет(((\
Из плюсов - модель ну совсем простая и работает быстро

In [0]:
! pip install sister

In [16]:
import sister
embedder = sister.MeanEmbedding(lang="en")

sentence = "I am a dog."
vector = embedder(sentence)  # 300-dim vector

Loading model...


In [0]:
import random
import math
import razdel
import torch
import numpy as np
from rouge import Rouge


from torch.utils import data


class ExtDataset(data.Dataset):
    def __init__(self, records, bpe_processor, lower=True, max_sentences=30, max_sentence_length=50, device=torch.device('cpu')):
        self.records = records
        self.num_samples = records.shape[0]
        self.bpe_processor = bpe_processor
        self.lower = lower
        self.rouge = Rouge()
        
        self.max_sentences = max_sentences
        self.max_sentence_length = max_sentence_length
        self.device = device
        
    def __len__(self):
        return self.records.shape[0]

    def __getitem__(self, idx):
        cur_record = self.records.iloc[idx]
        inputs = [self.bpe_processor(cur_record['sentences'][i]) for i in range(len(cur_record['sentences'][:self.max_sentences]))] 
        outputs = [int(i in cur_record['oracle_sentences']) for i in range(len(cur_record['sentences']))]
        return {'inputs': inputs, 'outputs': outputs}

In [0]:
# Это батчевалка
def collate_fn(records):
    max_length = max(len(sentence) for record in records for sentence in record['inputs'])
    max_sentences = max(len(record['outputs']) for record in records)

    new_inputs = torch.zeros((len(records), max_sentences, max_length))
    new_outputs = torch.zeros((len(records), max_sentences))
    for i, record in enumerate(records):
        for j, sentence in enumerate(record['inputs']):
            new_inputs[i, j, :len(sentence)] += np.array(sentence)
        new_outputs[i, :len(record['outputs'])] += np.array(record['outputs'])
    return {'features': new_inputs.type(torch.FloatTensor), 'targets': new_outputs}

In [29]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.nn.utils.rnn import pack_padded_sequence as pack
from torch.nn.utils.rnn import pad_packed_sequence as unpack


class SentenceEncoderRNN(nn.Module):
    def __init__(self, input_size, embedding_dim, hidden_size, n_layers=3, dropout=0.3, bidirectional=True):
        super(SentenceEncoderRNN, self).__init__()

        num_directions = 2 if bidirectional else 1
        assert hidden_size % num_directions == 0
        hidden_size = hidden_size // num_directions

        self.embedding_dim = embedding_dim
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.dropout = dropout
        self.bidirectional = bidirectional

        self.embedding_layer = nn.Embedding(input_size, embedding_dim)
        self.rnn_layer = nn.LSTM(embedding_dim, hidden_size, n_layers, dropout=dropout, bidirectional=bidirectional, batch_first=True)
        self.dropout_layer = nn.Dropout(dropout)

    def forward(self, inputs, hidden=None):
        embedded = self.embedding_layer(inputs)
        outputs, _ = self.rnn_layer(embedded, hidden)
        sentences_embeddings = torch.mean(outputs, 1)
        return sentences_embeddings


class YourSentenceEncoder(torch.nn.Module):
    # Место для вашего Sentence Encoder-а. Разрешается использовать любые методы, которые вам нравятся.
    # мне не супер нравится этот метод, но это самое простое, на что хватило времени(
    def __init__(self):
        super(YourSentenceEncoder, self).__init__()
        self.h_s = 300
        


    def forward(self, inputs):
      prosta_embedosy = inputs
      return prosta_embedosy
      



class SentenceTaggerRNN(nn.Module):
    def __init__(self,
                 vocabulary_size=9000,
                 token_embedding_dim=256,
                 sentence_encoder_hidden_size=300,
                 hidden_size=600,
                 bidirectional=True,
                 sentence_encoder_n_layers=2,
                 sentence_encoder_dropout=0.3,
                 sentence_encoder_bidirectional=True,
                 n_layers=1,
                 dropout=0.3):
        super(SentenceTaggerRNN, self).__init__()

        num_directions = 2 if bidirectional else 1
        assert hidden_size % num_directions == 0
        hidden_size = hidden_size // num_directions

        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.dropout = dropout
        self.bidirectional = bidirectional
        self.enc_hs = sentence_encoder_hidden_size

        self.sentence_encoder = YourSentenceEncoder()
        self.rnn_layer = nn.LSTM(sentence_encoder_hidden_size, hidden_size, n_layers, dropout=dropout,
                           bidirectional=bidirectional, batch_first=True)
        self.dropout_layer = nn.Dropout(dropout)
        self.one_more_lr = nn.Linear(sentence_encoder_hidden_size, hidden_size*2)
        self.content_linear_layer = nn.Linear(hidden_size*2, 1)
        self.document_linear_layer = nn.Linear(hidden_size*2, hidden_size*2)
        self.salience_linear_layer = nn.Linear(hidden_size*2, hidden_size*2)
        self.tanh_layer = nn.Tanh()
        self.soft_max = nn.Softmax(dim =1)

    def forward(self, inputs, hidden=None):
        

        batch_size = inputs.size(0)
        sentences_count = inputs.size(1)
        tokens_count = inputs.size(2)
        inputs = inputs.reshape(-1, tokens_count)
        embedded_sentences = self.sentence_encoder(inputs)
        embedded_sentences = embedded_sentences.reshape(batch_size, -1, self.enc_hs)
        outputs = self.dropout_layer(embedded_sentences)
        outputs = self.one_more_lr(outputs)
        document_embedding = self.tanh_layer(self.document_linear_layer(torch.mean(outputs, 1)))
        content = self.content_linear_layer(outputs).squeeze(2)
        salience = torch.bmm(outputs, self.salience_linear_layer(document_embedding).unsqueeze(2)).squeeze(2)
        res = self.soft_max(content + salience)

        return res

model = SentenceTaggerRNN()

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:50: UserWarning:

dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1



In [30]:
model.to(device)

SentenceTaggerRNN(
  (sentence_encoder): YourSentenceEncoder()
  (rnn_layer): LSTM(300, 300, batch_first=True, dropout=0.3, bidirectional=True)
  (dropout_layer): Dropout(p=0.3, inplace=False)
  (one_more_lr): Linear(in_features=300, out_features=600, bias=True)
  (content_linear_layer): Linear(in_features=600, out_features=1, bias=True)
  (document_linear_layer): Linear(in_features=600, out_features=600, bias=True)
  (salience_linear_layer): Linear(in_features=600, out_features=600, bias=True)
  (tanh_layer): Tanh()
  (soft_max): Softmax(dim=1)
)

### Обучение

In [31]:
import catalyst
from catalyst.dl.runner import SupervisedRunner

device = torch.device('cuda')

loaders = {
    'train': data.DataLoader(ExtDataset(ext_train_records,  bpe_processor=embedder), batch_size=16, collate_fn=collate_fn),
    'valid': data.DataLoader(ExtDataset(ext_val_records, bpe_processor=embedder), batch_size=16, collate_fn=collate_fn),
    'test': data.DataLoader(ExtDataset(ext_test_records, bpe_processor=embedder), batch_size=16, collate_fn=collate_fn),
}

lr = 5e-5
num_epochs = 10

optimizer  = torch.optim.Adam(model.parameters(), lr=lr)
criterion = nn.BCEWithLogitsLoss()
runner = SupervisedRunner()
runner.train(
    model=model,
    optimizer=optimizer,
    loaders=loaders,
    logdir='./logs',
    num_epochs=num_epochs,
    criterion=criterion,
    verbose=True
)


Streaming output truncated to the last 5000 lines.
2/10 * Epoch (train):  70% 178/256 [00:49<00:21,  3.62it/s, loss=0.707]
2/10 * Epoch (train):  70% 179/256 [00:49<00:20,  3.68it/s, loss=0.707]
2/10 * Epoch (train):  70% 179/256 [00:49<00:20,  3.68it/s, loss=0.707]
2/10 * Epoch (train):  70% 180/256 [00:49<00:20,  3.62it/s, loss=0.707]
2/10 * Epoch (train):  70% 180/256 [00:49<00:20,  3.62it/s, loss=0.707]
2/10 * Epoch (train):  71% 181/256 [00:49<00:20,  3.60it/s, loss=0.707]
2/10 * Epoch (train):  71% 181/256 [00:50<00:20,  3.60it/s, loss=0.708]
2/10 * Epoch (train):  71% 182/256 [00:50<00:20,  3.56it/s, loss=0.708]
2/10 * Epoch (train):  71% 182/256 [00:50<00:20,  3.56it/s, loss=0.707]
2/10 * Epoch (train):  71% 183/256 [00:50<00:20,  3.56it/s, loss=0.707]
2/10 * Epoch (train):  71% 183/256 [00:50<00:20,  3.56it/s, loss=0.708]
2/10 * Epoch (train):  72% 184/256 [00:50<00:19,  3.62it/s, loss=0.708]
2/10 * Epoch (train):  72% 184/256 [00:50<00:19,  3.62it/s, loss=0.708]
2/10 * Epoch 

In [32]:
device = torch.device("cuda")

references = []
predictions = []
model.eval()
for i, item in tqdm(enumerate(data.DataLoader(ExtDataset(ext_test_records, bpe_processor=embedder), batch_size=1, collate_fn=collate_fn)), total=ext_test_records.shape[0]):
    logits = model(item["features"].to(device))[0] # Прямой проход
    record = ext_test_records.iloc[i]
    predicted_summary = []
    for i, logit in enumerate(logits):
        if logit > 0.1:
            predicted_summary.append(record['sentences'][i])
    if not predicted_summary:
        predicted_summary.append(record['sentences'][torch.max(logits, dim=0)[1].item()])
    predicted_summary = " ".join(predicted_summary)
    references.append(record['summary'].lower())
    predictions.append(predicted_summary)

calc_scores(references, predictions)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning:

This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`




Count: 256
Ref: евразийский экономический союз прирастает сингапуром. с этой страной подписано соглашение о зоне свободной торговли. на подходе заключение подобных договоров с ираном, сербией и индией. таков итог саммита в ереване. но расширенный союз унаследует все болезни роста, которые не решаются уже много лет. и в первую очередь это торговые барьеры и отсутствие инвестиций.
Hyp: и такое соглашение было подписано во вторник 1 октября в ереване по итогам заседания высшего совета евразийского экономического союза пяти государств: россии, казахстана, белоруссии, киргизии и армении. россия же пребывает на 45-м месте в рейтинге. на самом деле он прибыл в ереван за три дня до начала саммита по приглашению премьера армении николы пашиняна. в июле никол пашинян побывал в сингапуре.
BLEU:  0.36690031480782226
ROUGE:  {'rouge-1': {'f': 0.1580674514886958, 'p': 0.17573252354973667, 'r': 0.16286442987000724}, 'rouge-2': {'f': 0.04509052884835112, 'p': 0.046718785242665296, 'r': 0.049593143900